In [1]:
#Import everything
from http import cookies
import selenium
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
import json
import urllib
import requests
import os
'''
def click_element(xpath: str):
    try:
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()
    except TimeoutException:
        print("TimeoutException: Button not found")
'''

'\ndef click_element(xpath: str):\n    try:\n        button = self.driver.find_element(By.XPATH, xpath)\n        button.click()\n    except TimeoutException:\n        print("TimeoutException: Button not found")\n'

In [2]:
# Open the website
driver = webdriver.Chrome()
driver.get('https://vndb.org/v')

In [3]:
# Try out next page
button = driver.find_element(By.XPATH, '/html/body/div[4]/form/div[2]/ul[2]/li[1]/a')
button.click()
# Yeah!!!

In [6]:
# Try out search field
search_field = driver.find_element(By.XPATH, '//*[@id="q"]')
search_field.clear()
search_field.send_keys('black')
search_button = driver.find_element(By.XPATH, '/html/body/div[4]/form/div[1]/fieldset/input[2]')
search_button.click()

In [7]:
# Get the first result in the search field
result_1 = driver.find_elements(By.XPATH, '/html/body/div[4]/form/div[3]/table/tbody/tr[3]/td[2]/img')
#print(result_1[3].get_attribute('title'))
table = driver.find_element(By.XPATH, '/html/body/div[4]/form/div[3]/table/tbody')
rows = table.find_elements(By.XPATH, './tr')
#print(rows[0].text)
row_1_elements = rows[0].find_elements(By.XPATH, './td')
print(row_1_elements[0].text)
print(row_1_elements[5].text.split()[1][1:-1])
# I find the columns to be:
# 1: Title of game
# 2: Platforms available (in images, need to be dealt with separately)
# 3: Languages available (in images, need to be dealt with separately)
# 4: Release date
# 5: Popularity index
# 6: Rating out of 10 (No. of users rated on VNDB)
platforms = row_1_elements[1].find_elements(By.XPATH, './img')
languages = row_1_elements[2].find_elements(By.XPATH, './abbr')
for platform in platforms:
    print(platform.get_attribute('title'))
for language in languages:
    print(language.get_attribute('title'))
# Get url to description page
description_page_url = row_1_elements[0].find_element(By.XPATH, './a').get_attribute('href')
print(description_page_url)

Akuma Shitsuji to Kuroi Neko
2
Android
Apple iProduct
Japanese
https://vndb.org/v33910


In [12]:
# Now get all useful information from the table and store 'em in a dictionary
table = driver.find_element(By.XPATH, '/html/body/div[4]/form/div[3]/table/tbody')
rows = table.find_elements(By.XPATH, './tr')
table_data = [] # going to be a list of dictionaries
for row in rows[:5]:
    row_elements = row.find_elements(By.XPATH, './td')
    title = row_elements[0].text
    release_date = row_elements[3].text
    popularity = row_elements[4].text
    rating = row_elements[5].text.split()[0]
    no_of_voters = int(row_elements[5].text.split()[1][1:-1]) # need to take away brackets

    # Platforms and languages are displayed in images, need to be dealt with separately
    platforms = []
    languages = []
    platforms_img = row_elements[1].find_elements(By.XPATH, './img')
    languages_img = row_elements[2].find_elements(By.XPATH, './abbr')
    for platform in platforms_img:
        platforms.append(platform.get_attribute('title'))
    platforms = ", ".join(platforms)
    for language in languages_img:
        languages.append(language.get_attribute('title'))
    languages = ", ".join(languages)
    # Get description page link
    description_page_url = row_elements[0].find_element(By.XPATH, './a').get_attribute('href')

    row_data_dict = {"URL": description_page_url,
                     "Title": title,
                     "Platform_available": platforms,
                     "Language_available": languages,
                     "Release_date": release_date,
                     "Popularity": popularity,
                     "Rating": rating,
                     "No_of_voters": no_of_voters}
    table_data.append(row_data_dict)
print(table_data[3])

 
    

{'URL': 'https://vndb.org/v33809', 'Title': 'a love colored blackbird', 'Platform_available': 'Linux, Mac OS, Windows', 'Language_available': 'English', 'Release_date': '2022-02-01', 'Popularity': '0.00', 'Rating': '0.00', 'No_of_voters': 0}


In [30]:
# Now we get the data, we store them
import pandas as pd
dataframe = pd.DataFrame(table_data)
dataframe.to_json('data.json')
print(dataframe)

                       URL                                     Title  \
0  https://vndb.org/v33910              Akuma Shitsuji to Kuroi Neko   
1  https://vndb.org/v25691               ALLBLACK Phase #1 -Project-   
2  https://vndb.org/v30910                ALLBLACK -Phase#2:Promise-   
3  https://vndb.org/v33809                  a love colored blackbird   
4    https://vndb.org/v995  Altered Pink ~Tokumu Sentai Duel Ranger~   

                         Platform_available  \
0                   Android, Apple iProduct   
1  Android, Linux, Mac OS, Website, Windows   
2                    Linux, Mac OS, Windows   
3                    Linux, Mac OS, Windows   
4                       DVD Player, Windows   

                    Language_available Release_date Popularity Rating  \
0                             Japanese   2021-12-07       0.02   6.26   
1  Russian, Japanese, Spanish, English   2019-04-21       0.11   6.20   
2                              English   2021-04-02       0.00   

In [29]:
# Error handling
from selenium.common.exceptions import TimeoutException, NoSuchElementException
try:
    button = driver.find_element(By.XPATH, '/html/body/div[4]/form/div[2]/ul[2]/li[1]/a')
    button.click()
except NoSuchElementException:
    print("Element not found!")

Element not found!
